In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

# statsmodels API 불러오기
import statsmodels.api as sm

# 분산 팽창 계수 함수 불러오기
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# 교차검증 횟수
cv_count = 10

# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

In [3]:
# 데이터 불러오기
train_df = pd.read_csv("final_train.csv")
test_df = pd.read_csv("final_test.csv")

train_df = train_df.drop(columns=['ID'])

df1 = train_df

In [4]:
# 입력 변수(X)와 타깃 변수(y)로 데이터를 분리한다.
X = df1.drop('Segment', axis=1)
y = df1['Segment']

# 문자열 -> 숫자
encoder1 = LabelEncoder()
encoder1.fit(y)
y2 = encoder1.transform(y)

# 입력 데이터 표준화
scaler1 = StandardScaler()
scaler1.fit(X)
X2 = scaler1.transform(X)

# 학습용과 검증용으로 나눈다.
train_X = X2
train_y = y2

### XGBoost

In [5]:
xgboost_basic_model = XGBClassifier(verbose=-1, verbosity=0, tree_method='gpu_hist', predictor = 'gpu_predictor')

# 모델 학습 수행
xgboost_basic_model.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [6]:
# 학습 시 사용한 피처 컬럼명 저장
feature_cols = df1.drop(columns=['Segment']).columns


# test 데이터 ID, Segment 제거 후 학습 컬럼 순서에 맞게 재정렬
test_ids = test_df['ID']
test_features = test_df.drop(columns=['ID', 'Segment'], errors='ignore')
test_features = test_features.loc[:, feature_cols]

# 입력 데이터 스케일링
test_scaled = scaler1.transform(test_features)

In [7]:
# 모델 예측 수행
pred_y_numeric = xgboost_basic_model.predict(test_scaled)

# 예측 결과 클래스별 분포 확인
unique_vals_num, counts_num = np.unique(pred_y_numeric, return_counts=True)
print(dict(zip(unique_vals_num, counts_num)))

{0: 10, 2: 4217, 3: 13567, 4: 82206}


In [8]:
# 숫자형 예측 결과를 원래 클래스 레이블로 변환
pred_y = encoder1.inverse_transform(pred_y_numeric)

# 역변환된 예측 결과 분포 출력
unique_vals_str, counts_str = np.unique(pred_y, return_counts=True)
print(dict(zip(unique_vals_str, counts_str)))

{'A': 10, 'C': 4217, 'D': 13567, 'E': 82206}


In [9]:
# 예측 결과를 데이터프레임에 추가하고 중복 ID 제거
submission_df = pd.DataFrame({'ID': test_ids, 'Segment': pred_y})
submission_df = submission_df.drop_duplicates(subset='ID')

In [10]:
# 샘플 제출 파일 불러오기
submit = pd.read_csv('open/sample_submission.csv')

In [11]:
# 예측 결과로 대체
submit['Segment'] = submission_df.set_index('ID').reindex(submit['ID'])['Segment'].values

# 결과를 CSV 파일로 출력
submit.to_csv('최종모델.csv', index=False)

In [12]:
# 최종 제출 파일 내용 출력
submit

,ID,Segment
0,TEST_00000,D
1,TEST_00001,E
2,TEST_00002,E
3,TEST_00003,E
4,TEST_00004,E
...,...,...
99995,TEST_99995,E
99996,TEST_99996,E
99997,TEST_99997,E
99998,TEST_99998,C
